## 对上周学习目标的回顾
1. 使用 requests-html 爬取并存取网页文字档，查找requests-html 中文文档
2. 熟悉 xpath 语法丶xpath 节点
3. 使用 xpath cheatsheet
    * 在 Chrome Inspector 使用
    * 在 requests-html (Python) 使用
4. 简易使用 pd.DataFrame

### 猎聘移动端(代码由老师提供，本人添加注释)

In [1]:
# 基本模块
import pandas as pd
from requests_html import HTMLSession

In [2]:
# C-1   单一页面
url = "https://m.liepin.com/zhaopin/?keyword=%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90%E5%B8%88"
from requests_html import HTMLSession
session = HTMLSession()
r = session.get( url )

# C-5
# 难: '公司URL', '时间', '经验'

# 先取特定元素, 精准打击其子后辈
主要元素 = r.html.xpath( \
    '//div[@class="job-card-wrap"]//div[@class="job-card"]')

# 作为xpath字典，键为我要抓的牛肉名称，值为xpath
dict_xpaths={ 
    'text': {
        '经验':      './/ul/li[time]/text()'
    },
    'text_content': {
        '职称':    './/ul/li/a[contains(@class,"job-name")]/span[@class="name-text"]', 
        '薪水':    './/ul/li/a[contains(@class,"job-name")]/following-sibling::span', #获取弟节点的方法：following-sibling::元素名
        '公司地点':'.//ul/li/time/following-sibling::a',                              #获取兄节点的方法：preceding-sibling::元素名
        '公司名称': './/ul/li/a[contains(@class,"company-name")]', 
        '时间':    './/ul/li/time', 
    },
    'href': {
        '链结':    './/ul/li/a[contains(@class,"job-name")]', 
        '公司URL': './/ul/li/a[contains(@class,"company-name")]', #在class属性不止一个时，利用[contains(@class,"指定class属性值")]定位
    }
}

def get_e_text_content(_xpath_):
    # 高级列表推导
    暂存结果 = [e.xpath(_xpath_)[0].lxml.text_content() for e in 主要元素]   
    return(暂存结果)

def get_e_text(_xpath_):
    # 高级列表推导
    暂存结果 = ["".join([x.strip() for x in e.xpath(_xpath_)]) for e in 主要元素]
    return(暂存结果)

def get_e_href(_xpath_):
    # 高级列表推导:如果URL存在（即len(xxx)>=1）则输出列表，否则输出一个空字符串
    暂存结果 = [list(e.xpath(_xpath_, first=True).absolute_links)[0]\
               if len(e.xpath(_xpath_, first=True).absolute_links) >=1\
               else "" for e in 主要元素]
    return(暂存结果)

# 只对主要元素下进行.xpath取值
数据字典 = dict()

数据字典 = {k:get_e_text_content(v) for k,v in dict_xpaths['text_content'].items()}

数据字典.update({k:get_e_text(v) for k,v in dict_xpaths['text'].items()})
数据字典.update({k:get_e_href(v) for k,v in dict_xpaths['href'].items()})
数据字典
#print ([len(v) for k,v in 数据字典.items()])  # 檢查

数据 = pd.DataFrame(数据字典)
#数据.to_excel("20春_Web数据挖掘_week02_liepin.xlsx", sheet_name="搜查结果")
数据 

,职称,薪水,公司地点,公司名称,时间,经验,链结,公司URL
0,数据分析师,15-23k·13薪,广州-天河区,数果智能,10小时前,3年以上 本科及以上,https://m.liepin.com/job/1926363353.shtml,https://m.liepin.com/company/8837861/
1,数据分析师,面议,广州,如祺出行,昨天,3年以上 统招本科,https://m.liepin.com/job/1927117161.shtml,https://m.liepin.com/company/10094945/
2,数据分析师,10-16k·12薪,广州-新市,广州伊羽霓贸易有限公司,昨天,2年以上 大专及以上,https://m.liepin.com/job/1926600635.shtml,https://m.liepin.com/company/9705250/
3,数据分析师,18-33k·12薪,广州,卓越教育集团,昨天,5年以上 本科及以上,https://m.liepin.com/job/1926567413.shtml,https://m.liepin.com/company/876848/
4,数据分析师,8-15k·12薪,广州,大麦信息,昨天,3年以上 本科及以上,https://m.liepin.com/job/1926420031.shtml,https://m.liepin.com/company/7870705/
5,数据分析师,15-25k·12薪,广州-番禺区,广州探迹科技有限公司,昨天,5年以上 统招本科,https://m.liepin.com/job/1926292629.shtml,https://m.liepin.com/company/8836020/
6,数据分析师,6-8k·12薪,广州,嫚熙控股(广州)有限公司,昨天,3年以上 大专及以上,https://m.liepin.com/job/1925238817.shtml,https://m.liepin.com/company/9660700/
7,数据分析师,20-35k·15薪,"北京,广州,上海",xx公司,2020-03-26,2年以上 学历不限,https://m.liepin.com/a/18569421.shtml,
8,数据分析师,8-12k·13薪,广州-天河北,维他奶,昨天,1年以上 统招本科,https://m.liepin.com/job/1924641017.shtml,https://m.liepin.com/company/8428035/
9,数据分析师,面议,广州-天河北,租租车,昨天,1年以上 统招本科,https://m.liepin.com/job/1921434973.shtml,https://m.liepin.com/company/8082331/


## 本周目标¶
* 猎聘PC版
* 上方导航有 公司行业 城市 薪资 的分页选单
* 请练习xpath抽出数据

### 爬取猎聘PC版职位，薪水，公司等数据

In [55]:
from requests_html import HTMLSession
import pandas as pd
import lxml
session= HTMLSession()

url="https://www.liepin.com/zhaopin/?industries=&subIndustry=&dqs=&salary=&jobKind=&pubTime=&compkind=&compscale=&industryType=&searchType=1&clean_condition=&isAnalysis=&init=1&sortFlag=15&flushckid=0&fromSearchBtn=1&headckid=52b1c0be834fc722&d_headId=41c0cceeae9b7e1f5aec9cc84ba87032&d_ckId=41c0cceeae9b7e1f5aec9cc84ba87032&d_sfrom=search_prime&d_curPage=0&d_pageSize=40&siTag=bFGQTbwE_AAQSb-u11jrBw%7EfA9rXquZc5IkJpXC-Ycixw&key=%E6%95%B0%E6%8D%AE%E6%8C%96%E6%8E%98"
r= session.get(url)
#设置主要的xpath路径
main= r.html.xpath("//div[contains(@class,'sojob-item-main')]")
#使用字典存取特定的数据的位置
dict_xpath={
    "text":{
        "职位":".//div[@class='job-info']/h3/a/text()"
    },
    "text_content":{
        "时间":".//div[@class='job-info']/p[contains(@class,'time-info')]/time/@title",
        
        "薪水":".//div[@class='job-info']/p/span[@class='text-warning']/text()",
        "工作地点":".//p[contains(@class,condition)]/a/text()",  #/div[@class='job-info']      [@class='area']
        "学历":".//span[@class='edu']/text()",
        "经验":".//p[contains(@class,condition)]/span[3]/text()",
        "公司名称":".//p[contains(@class,'company-name')]/a/text()"
    },
    "href":{
        "职位链接":".//div[@class='job-info']/h3/a",
        "公司链接":"//div[contains(@class,'sojob-item-main')]//p[@class='company-name']/a" #/div[contains(@class,'company-info')]
    }
    
}

def get_e_text(_xpath_):
    # 高级列表推导
    暂存结果 = ["".join([x.strip() for x in e.xpath(_xpath_)]) for e in main]
    return(暂存结果)

def get_text_content(_xpath_):
    # 高级列表推导
    
    暂存结果 =[ e.xpath(_xpath_)[0]  for e in main]
    
    #暂存结果 =main#.xpath(_xpath_)
    return(暂存结果)          

#如果为真，first=True返回第一个元素

def get_href(_xpath_):          
    暂存结果=[list(e.xpath(_xpath_)[0].absolute_links)[0]              
                  if len(list(e.xpath(_xpath_)[0].absolute_links))>=1
                        else ""  for e in main  ]
          

    return(暂存结果)
数据字典=dict()
数据字典={k:get_text_content(v) for k,v in dict_xpath["text_content"].items()}
数据字典.update({k:get_e_text(v) for k,v in dict_xpath['text'].items()})
数据字典.update({k:get_href(v) for k,v in dict_xpath['href'].items() })
数据字典
df=pd.DataFrame(数据字典)
df
#with open('猎聘网数据挖掘方面招聘信息.xlsx' ,encoding = "utf8", mode = 'a') as fp:
#    df.to_excel("猎聘网数据挖掘方面招聘信息.xlsx", sheet_name="猎聘PC")
#数据
#eweqw ={k:get_href(v) for k,v in dict_xpath["href"].items() }
#eweqw
#for k,v in dict_xpath["text"].items():
#       print(k,get_e_text(v) )
#for k,v in dict_xpath["href"].items():
#       print(k,get_href(v) )

,时间,薪水,工作地点,学历,经验,公司名称,职位,职位链接,公司链接
0,2020年04月03日,10-18k·12薪,广州,本科及以上,2年以上,时代大数据,数据挖掘工程师,https://www.liepin.com/job/1920605639.shtml,https://www.liepin.com/company/9382271/
1,2020年04月05日,面议,美团点评,本科及以上,本科及以上,美团点评,数据中心-数据挖掘算法工程师,https://www.liepin.com/job/1925789257.shtml,https://www.liepin.com/company/2036768/
2,2020年04月05日,10-15k·12薪,深演智能,统招本科,统招本科,深演智能,2020校园招聘-数据挖掘工程师,https://www.liepin.com/job/1923453745.shtml,https://www.liepin.com/company/1157027/
3,2020年04月05日,面议,上海-长宁区,本科及以上,3年以上,美团点评,点评App部-机器学习/数据挖掘算法专家,https://www.liepin.com/job/1923289363.shtml,https://www.liepin.com/company/2036768/
4,2020年04月05日,面议,美团点评,本科及以上,本科及以上,美团点评,闪购-数据挖掘技术专家,https://www.liepin.com/job/1919432959.shtml,https://www.liepin.com/company/2036768/
5,2020年04月05日,20-40k·13薪,北京,硕士及以上,3年以上,深演智能,高级数据挖掘工程师,https://www.liepin.com/job/1918060629.shtml,https://www.liepin.com/company/1157027/
6,2020年04月05日,面议,美团点评,本科及以上,本科及以上,美团点评,美团外卖-数据挖掘技术专家,https://www.liepin.com/job/1917876419.shtml,https://www.liepin.com/company/2036768/
7,2020年04月05日,25-35k·14薪,kika公司,统招本科,统招本科,kika公司,数据挖掘工程师,https://www.liepin.com/a/19593545.shtml,
8,2020年04月05日,30-50k·16薪,深圳-南山区,统招本科,3年以上,某综合性集团公司,数据挖掘工程师,https://www.liepin.com/a/19577517.shtml,
9,2020年04月05日,16-20k·15薪,深圳,统招本科,2年以上,国内知名金融保险公司,数据挖掘,https://www.liepin.com/a/19468911.shtml,


## 使用urllib3 解析 url¶
* 上面的url应该触动不同的页面查询，但能不能轻松无误的拆分url并进行比较？

### urllib模块功能介绍¶
* urlparse 返回的6个部分，分别是：scheme(机制)丶netloc(网络位置)丶path(路径)丶params(路径段参数)丶query(查询)丶fragment(片段)。
parse_qs 返回query(查询)多个部分

In [10]:
#使用 urllib.parse 解析
from urllib.parse import urlparse, parse_qs

### 取出特定URL

In [27]:
company=r.html.xpath('//div[@data-selector="search-conditions"]')[0] \
                    .xpath('//dt[@class="search-title"]/following-sibling::dd')[0] \
                    .xpath('//div[contains(@class,"hot-comp-tags")]/a')
company_lianjie={x.xpath('a/text()')[0]:x.xpath('a/@href')[0]  for x in company}  #输出的是一个列表，[0]取出值
company_lianjie

{'中国500强': '/zhaopin/?isAnalysis=&dqs=&pubTime=&salary=&subIndustry=&industryType=&compscale=&key=%E6%95%B0%E6%8D%AE%E6%8C%96%E6%8E%98&init=-1&searchType=1&headckid=970029fdeea53da8&flushckid=1&compkind=&fromSearchBtn=2&compTag=155&sortFlag=15&ckid=970029fdeea53da8&jobKind=&industries=&clean_condition=&siTag=LiAE77uh7ygbLjiB5afMYg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_prime&d_ckId=5b217fbd8e34ad6a68b83bd1bebc133f&d_curPage=0&d_pageSize=40&d_headId=5b217fbd8e34ad6a68b83bd1bebc133f',
 '2018互联网300强': '/zhaopin/?isAnalysis=&dqs=&pubTime=&salary=&subIndustry=&industryType=&compscale=&key=%E6%95%B0%E6%8D%AE%E6%8C%96%E6%8E%98&init=-1&searchType=1&headckid=970029fdeea53da8&flushckid=1&compkind=&fromSearchBtn=2&compTag=182&sortFlag=15&ckid=970029fdeea53da8&jobKind=&industries=&clean_condition=&siTag=LiAE77uh7ygbLjiB5afMYg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_prime&d_ckId=5b217fbd8e34ad6a68b83bd1bebc133f&d_curPage=0&d_pageSize=40&d_headId=5b217fbd8e34ad6a68b83bd1bebc133f',
 '制造业500强': '/zha

### 分解URL

In [28]:
#取出键company_lianjie的值，利用for循环对所有url进行拆解  
[ urlparse(x) for x in company_lianjie.values()]

[ParseResult(scheme='', netloc='', path='/zhaopin/', params='', query='isAnalysis=&dqs=&pubTime=&salary=&subIndustry=&industryType=&compscale=&key=%E6%95%B0%E6%8D%AE%E6%8C%96%E6%8E%98&init=-1&searchType=1&headckid=970029fdeea53da8&flushckid=1&compkind=&fromSearchBtn=2&compTag=155&sortFlag=15&ckid=970029fdeea53da8&jobKind=&industries=&clean_condition=&siTag=LiAE77uh7ygbLjiB5afMYg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_prime&d_ckId=5b217fbd8e34ad6a68b83bd1bebc133f&d_curPage=0&d_pageSize=40&d_headId=5b217fbd8e34ad6a68b83bd1bebc133f', fragment=''),
 ParseResult(scheme='', netloc='', path='/zhaopin/', params='', query='isAnalysis=&dqs=&pubTime=&salary=&subIndustry=&industryType=&compscale=&key=%E6%95%B0%E6%8D%AE%E6%8C%96%E6%8E%98&init=-1&searchType=1&headckid=970029fdeea53da8&flushckid=1&compkind=&fromSearchBtn=2&compTag=182&sortFlag=15&ckid=970029fdeea53da8&jobKind=&industries=&clean_condition=&siTag=LiAE77uh7ygbLjiB5afMYg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_prime&d_ckId=5b217fbd8e34a

### nunique()相异值计量比对
* 使用 pd.DataFrame进行 nunique()相异值计量比对  
* Pandas nunique() 用于获取唯一值的统计次数。

In [30]:
df=pd.DataFrame([ urlparse(x) for x in company_lianjie.values()])
df

,scheme,netloc,path,params,query,fragment
0,,,/zhaopin/,,isAnalysis=&dqs=&pubTime=&salary=&subIndustry=...,
1,,,/zhaopin/,,isAnalysis=&dqs=&pubTime=&salary=&subIndustry=...,
2,,,/zhaopin/,,isAnalysis=&dqs=&pubTime=&salary=&subIndustry=...,
3,,,/zhaopin/,,isAnalysis=&dqs=&pubTime=&salary=&subIndustry=...,
4,,,/zhaopin/,,isAnalysis=&dqs=&pubTime=&salary=&subIndustry=...,
5,,,/zhaopin/,,isAnalysis=&dqs=&pubTime=&salary=&subIndustry=...,


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   scheme    6 non-null      object
 1   netloc    6 non-null      object
 2   path      6 non-null      object
 3   params    6 non-null      object
 4   query     6 non-null      object
 5   fragment  6 non-null      object
dtypes: object(6)
memory usage: 416.0+ bytes


In [32]:
df.nunique()

scheme      1
netloc      1
path        1
params      1
query       6
fragment    1
dtype: int64

###  针对query 再解析之 
* urllib.parse.parse_qs 返回字典 ,其值为列表
* urllib.parse.parse_qsl 返回列表

In [48]:
#利用parse_qs进行解析
#for循环取出query的字符串，再用字典推导式，parse_qs把url解析成列表，键为索引，值为url   
df_qs=pd.DataFrame([{k:v[0] for k,v in parse_qs(e).items()}for e in df['query']])
df_qs
print(df_qs.nunique())
df_qs[['key','compTag']]

key              1
init             1
searchType       1
headckid         1
flushckid        1
fromSearchBtn    1
compTag          6
sortFlag         1
ckid             1
siTag            1
d_sfrom          1
d_ckId           1
d_curPage        1
d_pageSize       1
d_headId         1
dtype: int64


,key,compTag
0,数据挖掘,155
1,数据挖掘,182
2,数据挖掘,186
3,数据挖掘,189
4,数据挖掘,130
5,数据挖掘,156


###  小结
* comTag 是不同的公司选择器, 数值不样, 对映到不同类型的公司
* keyword 是搜查关键字

### 建构 参数模板 及 字典_compTag


In [181]:
def parse_url_qs_for_compTag (url):
    six_parts = urlparse(url)    #利用urlparse分解url
    out = parse_qs(six_parts.query)#利用parse_qs拆解query的属性值
    return (out)
# parse_url_qs_for_compTag(list(公司数据选择器链结.values())[0])['compTag']
参数模板 = parse_url_qs_for_compTag(list(company_lianjie.values())[0])#取url的字符串
print(参数模板)

{'key': ['数据挖掘'], 'init': ['-1'], 'searchType': ['1'], 'headckid': ['970029fdeea53da8'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'compTag': ['155'], 'sortFlag': ['15'], 'ckid': ['970029fdeea53da8'], 'siTag': ['LiAE77uh7ygbLjiB5afMYg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_prime'], 'd_ckId': ['5b217fbd8e34ad6a68b83bd1bebc133f'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['5b217fbd8e34ad6a68b83bd1bebc133f']}


In [182]:
字典_compTag = { k:parse_url_qs_for_compTag(v)['compTag'][0] for k,v in company_lianjie.items()}#把url进行拆解，然后取出键为compTag的值，形成新的字典
print (字典_compTag)

{'中国500强': '155', '2018互联网300强': '182', '制造业500强': '186', 'AI创新成长50强 ': '189', '独角兽': '130', '上市公司': '156'}


In [183]:
# 建构 参数模板  
import pprint
def 参数模板生成(compTag , keyword ):
    参数 = 参数模板.copy()    #对url的query进行拆解
    参数['compTag'] = compTag#形式参数
    参数['keyword'] = keyword#形式参数
    return (参数)
#compTag = [v], keyword = ['用户体验']   compTag，keyword是形参
参数_compTag_用户体验 = { k:参数模板生成(compTag = [v], keyword = ['用户体验']) for k,v in 字典_compTag.items()}
pprint.pprint(参数_compTag_用户体验)               

{'2018互联网300强': {'ckid': ['970029fdeea53da8'],
                 'compTag': ['182'],
                 'd_ckId': ['5b217fbd8e34ad6a68b83bd1bebc133f'],
                 'd_curPage': ['0'],
                 'd_headId': ['5b217fbd8e34ad6a68b83bd1bebc133f'],
                 'd_pageSize': ['40'],
                 'd_sfrom': ['search_prime'],
                 'flushckid': ['1'],
                 'fromSearchBtn': ['2'],
                 'headckid': ['970029fdeea53da8'],
                 'init': ['-1'],
                 'key': ['数据挖掘'],
                 'keyword': ['用户体验'],
                 'searchType': ['1'],
                 'siTag': ['LiAE77uh7ygbLjiB5afMYg~fA9rXquZc5IkJpXC-Ycixw'],
                 'sortFlag': ['15']},
 'AI创新成长50强 ': {'ckid': ['970029fdeea53da8'],
                'compTag': ['189'],
                'd_ckId': ['5b217fbd8e34ad6a68b83bd1bebc133f'],
                'd_curPage': ['0'],
                'd_headId': ['5b217fbd8e34ad6a68b83bd1bebc133f'],
                'd_pageSize

## requests生成

In [184]:
#多个页面准备测试1 世界500强
url = "https://www.liepin.com/zhaopin/"
session = HTMLSession()
payload = 参数_compTag_用户体验['上市公司']
print(payload)
r = session.get( url, params = payload)
r.url

{'key': ['数据挖掘'], 'init': ['-1'], 'searchType': ['1'], 'headckid': ['970029fdeea53da8'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'compTag': ['156'], 'sortFlag': ['15'], 'ckid': ['970029fdeea53da8'], 'siTag': ['LiAE77uh7ygbLjiB5afMYg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_prime'], 'd_ckId': ['5b217fbd8e34ad6a68b83bd1bebc133f'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['5b217fbd8e34ad6a68b83bd1bebc133f'], 'keyword': ['用户体验']}


'https://www.liepin.com/zhaopin/?key=%E6%95%B0%E6%8D%AE%E6%8C%96%E6%8E%98&init=-1&searchType=1&headckid=970029fdeea53da8&flushckid=1&fromSearchBtn=2&compTag=156&sortFlag=15&ckid=970029fdeea53da8&siTag=LiAE77uh7ygbLjiB5afMYg~fA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_prime&d_ckId=5b217fbd8e34ad6a68b83bd1bebc133f&d_curPage=0&d_pageSize=40&d_headId=5b217fbd8e34ad6a68b83bd1bebc133f&keyword=%E7%94%A8%E6%88%B7%E4%BD%93%E9%AA%8C'

## 使用函数进行数据爬取

### 单个页面的爬取

In [185]:
import pandas as pd
from requests_html import HTMLSession
session= HTMLSession()

#url="https://www.liepin.com/zhaopin/"

#爬取主函数
def requests(url,params):
    r= session.get(url,params = payload)
    #设置主要的xpath路径
    main= r.html.xpath("//div[contains(@class,'sojob-item-main')]")
    #使用字典存取特定的数据的位置
    dict_xpath={
        "text":{
            "职位":".//div[@class='job-info']/h3/a/text()"
        },
        "text_content":{
            "时间":".//div[@class='job-info']/p[contains(@class,'time-info')]/time/@title",

            "薪水":".//div[@class='job-info']/p/span[@class='text-warning']/text()",
            "工作地点":".//p[contains(@class,condition)]/a/text()",  #/div[@class='job-info']      [@class='area']
            "学历":".//span[@class='edu']/text()",
            "经验":".//p[contains(@class,condition)]/span[3]/text()",
            "公司名称":".//p[contains(@class,'company-name')]/a/text()"
        },
        "href":{
            "职位链接":".//div[@class='job-info']/h3/a",
            "公司链接":"//div[contains(@class,'sojob-item-main')]//p[@class='company-name']/a" #/div[contains(@class,'company-info')]
        }

    }

def get_e_text(_xpath_):
        # 高级列表推导
        暂存结果 = ["".join([x.strip() for x in e.xpath(_xpath_)]) for e in main]
        return(暂存结果)

def get_text_content(_xpath_):
        # 高级列表推导

        暂存结果 =[ e.xpath(_xpath_)[0]  for e in main]

        #暂存结果 =main#.xpath(_xpath_)
        return(暂存结果)          

    #如果为真，first=True返回第一个元素

def get_href(_xpath_):          
        暂存结果=[list(e.xpath(_xpath_)[0].absolute_links)[0]              
                      if len(list(e.xpath(_xpath_)[0].absolute_links))>=1
                            else ""  for e in main  ]


        return(暂存结果)
数据字典=dict()
数据字典={k:get_text_content(v) for k,v in dict_xpath["text_content"].items()}
数据字典.update({k:get_e_text(v) for k,v in dict_xpath['text'].items()})
数据字典.update({k:get_href(v) for k,v in dict_xpath['href'].items() })
数据字典
df=pd.DataFrame(数据字典)
df

,时间,薪水,工作地点,学历,经验,公司名称,职位,职位链接,公司链接
0,2020年04月03日,10-18k·12薪,广州,本科及以上,2年以上,时代大数据,数据挖掘工程师,https://www.liepin.com/job/1920605639.shtml,https://www.liepin.com/company/9382271/
1,2020年04月05日,面议,美团点评,本科及以上,本科及以上,美团点评,数据中心-数据挖掘算法工程师,https://www.liepin.com/job/1925789257.shtml,https://www.liepin.com/company/2036768/
2,2020年04月05日,10-15k·12薪,深演智能,统招本科,统招本科,深演智能,2020校园招聘-数据挖掘工程师,https://www.liepin.com/job/1923453745.shtml,https://www.liepin.com/company/1157027/
3,2020年04月05日,面议,上海-长宁区,本科及以上,3年以上,美团点评,点评App部-机器学习/数据挖掘算法专家,https://www.liepin.com/job/1923289363.shtml,https://www.liepin.com/company/2036768/
4,2020年04月05日,面议,美团点评,本科及以上,本科及以上,美团点评,闪购-数据挖掘技术专家,https://www.liepin.com/job/1919432959.shtml,https://www.liepin.com/company/2036768/
5,2020年04月05日,20-40k·13薪,北京,硕士及以上,3年以上,深演智能,高级数据挖掘工程师,https://www.liepin.com/job/1918060629.shtml,https://www.liepin.com/company/1157027/
6,2020年04月05日,面议,美团点评,本科及以上,本科及以上,美团点评,美团外卖-数据挖掘技术专家,https://www.liepin.com/job/1917876419.shtml,https://www.liepin.com/company/2036768/
7,2020年04月05日,25-35k·14薪,kika公司,统招本科,统招本科,kika公司,数据挖掘工程师,https://www.liepin.com/a/19593545.shtml,
8,2020年04月05日,30-50k·16薪,深圳-南山区,统招本科,3年以上,某综合性集团公司,数据挖掘工程师,https://www.liepin.com/a/19577517.shtml,
9,2020年04月05日,16-20k·15薪,深圳,统招本科,2年以上,国内知名金融保险公司,数据挖掘,https://www.liepin.com/a/19468911.shtml,


### 多个页面进行爬取

In [186]:
import pandas as pd
from requests_html import HTMLSession
session= HTMLSession()

#url="https://www.liepin.com/zhaopin/"

#爬取主函数
def requests(url,params):
    r= session.get(url,params = payload)
    #设置主要的xpath路径
    main= r.html.xpath("//div[contains(@class,'sojob-item-main')]")
    #使用字典存取特定的数据的位置
    dict_xpath={
        "text":{
            "职位":".//div[@class='job-info']/h3/a/text()"
        },
        "text_content":{
            "时间":".//div[@class='job-info']/p[contains(@class,'time-info')]/time/@title",

            "薪水":".//div[@class='job-info']/p/span[@class='text-warning']/text()",
            "工作地点":".//p[contains(@class,condition)]/a/text()",  #/div[@class='job-info']      [@class='area']
            "学历":".//span[@class='edu']/text()",
            "经验":".//p[contains(@class,condition)]/span[3]/text()",
            "公司名称":".//p[contains(@class,'company-name')]/a/text()"
        },
        "href":{
            "职位链接":".//div[@class='job-info']/h3/a",
            "公司链接":"//div[contains(@class,'sojob-item-main')]//p[@class='company-name']/a" #/div[contains(@class,'company-info')]
        }

    }

    def get_e_text(_xpath_):
            # 高级列表推导
            暂存结果 = ["".join([x.strip() for x in e.xpath(_xpath_)]) for e in main]
            return(暂存结果)

    def get_text_content(_xpath_):
            # 高级列表推导

            暂存结果 =[ e.xpath(_xpath_)[0]  for e in main]

            #暂存结果 =main#.xpath(_xpath_)
            return(暂存结果)          

        #如果为真，first=True返回第一个元素

    def get_href(_xpath_):          
            暂存结果=[list(e.xpath(_xpath_)[0].absolute_links)[0]              
                          if len(list(e.xpath(_xpath_)[0].absolute_links))>=1
                                else ""  for e in main  ]


            return(暂存结果)
    数据字典=dict()
    数据字典={k:get_text_content(v) for k,v in dict_xpath["text_content"].items()}
    数据字典.update({k:get_e_text(v) for k,v in dict_xpath['text'].items()})
    数据字典.update({k:get_href(v) for k,v in dict_xpath['href'].items() })
    数据字典
    df=pd.DataFrame(数据字典)
    return(df)

In [187]:
#   多个页面,即多个公司
import pandas as pd
url = "https://www.liepin.com/zhaopin/"
#payload = 参数_compTag_用户体验['中国500强']
#DataFrame.assign(**kwargs)为DataFrame分配新列。返回一个新对象，该对象包含除新列之外的所有原始列。
list_ef = list()
#keyword为用户体验，key为数据，职位囊括上市公司，中国500强的公司的爬取
for k,v in 参数_compTag_用户体验.items():     
    payload = v
    ef = requests( url, params = payload)#调用requests_liepin()函数
    ef = df.assign(热门公司类型 = k)    
    list_ef.append(ef)
    
数据挖掘 = pd.concat(list_ef)  #对数据的合并与重塑
数据挖掘
#payload


,时间,薪水,工作地点,学历,经验,公司名称,职位,职位链接,公司链接,热门公司类型
0,2020年04月03日,10-18k·12薪,广州,本科及以上,2年以上,时代大数据,数据挖掘工程师,https://www.liepin.com/job/1920605639.shtml,https://www.liepin.com/company/9382271/,中国500强
1,2020年04月05日,面议,美团点评,本科及以上,本科及以上,美团点评,数据中心-数据挖掘算法工程师,https://www.liepin.com/job/1925789257.shtml,https://www.liepin.com/company/2036768/,中国500强
2,2020年04月05日,10-15k·12薪,深演智能,统招本科,统招本科,深演智能,2020校园招聘-数据挖掘工程师,https://www.liepin.com/job/1923453745.shtml,https://www.liepin.com/company/1157027/,中国500强
3,2020年04月05日,面议,上海-长宁区,本科及以上,3年以上,美团点评,点评App部-机器学习/数据挖掘算法专家,https://www.liepin.com/job/1923289363.shtml,https://www.liepin.com/company/2036768/,中国500强
4,2020年04月05日,面议,美团点评,本科及以上,本科及以上,美团点评,闪购-数据挖掘技术专家,https://www.liepin.com/job/1919432959.shtml,https://www.liepin.com/company/2036768/,中国500强
...,...,...,...,...,...,...,...,...,...,...
35,2020年04月04日,70-100k·12薪,深圳,统招本科,5年以上,卓尔人人,数据挖掘工程师,https://www.liepin.com/job/1925844509.shtml,https://www.liepin.com/company/12146335/,上市公司
36,2020年04月04日,70-100k·12薪,武汉,统招本科,5年以上,卓尔人人,数据挖掘工程师,https://www.liepin.com/job/1925844507.shtml,https://www.liepin.com/company/12146335/,上市公司
37,2020年04月04日,70-100k·12薪,上海,统招本科,5年以上,卓尔人人,数据挖掘工程师,https://www.liepin.com/job/1925844505.shtml,https://www.liepin.com/company/12146335/,上市公司
38,2020年04月04日,70-100k·12薪,北京,统招本科,5年以上,卓尔人人,数据挖掘工程师,https://www.liepin.com/job/1925844503.shtml,https://www.liepin.com/company/12146335/,上市公司


## 代码规整(重新练习)

### 建立session及取特定url

In [115]:
#建立会话
from requests_html import HTMLSession
session = HTMLSession()
#url填指定要爬取的页面
url="https://www.liepin.com/zhaopin/?industries=&subIndustry=&dqs=&salary=&jobKind=&pubTime=&compkind=&compscale=&industryType=&searchType=1&clean_condition=&isAnalysis=&init=1&sortFlag=15&flushckid=0&fromSearchBtn=1&headckid=52b1c0be834fc722&d_headId=41c0cceeae9b7e1f5aec9cc84ba87032&d_ckId=41c0cceeae9b7e1f5aec9cc84ba87032&d_sfrom=search_prime&d_curPage=0&d_pageSize=40&siTag=bFGQTbwE_AAQSb-u11jrBw%7EfA9rXquZc5IkJpXC-Ycixw&key=%E6%95%B0%E6%8D%AE%E6%8C%96%E6%8E%98"
r = session.get( url )
#获取url(筛选条件)
all_url=r.html.xpath('//div[contains(@class,"search-conditions")]')[0]\
                .xpath('//dt[@class="search-title"]/following-sibling::dd')[0]\
                .xpath('//dd[@class="comp-list"]/ul/li')


In [117]:
company_url={ x.xpath("a/span/text()")[0]:x.xpath("a/@href")[0] for x in all_url}
company_url

{'阿里巴巴': '/zhaopin/?isAnalysis=&dqs=&pubTime=&salary=&compIds=1072424&subIndustry=&industryType=&compscale=&key=%E6%95%B0%E6%8D%AE%E6%8C%96%E6%8E%98&init=-1&searchType=1&headckid=96731afb8363a6ab&flushckid=1&compkind=&fromSearchBtn=2&sortFlag=15&ckid=96731afb8363a6ab&jobKind=&industries=&clean_condition=&siTag=LiAE77uh7ygbLjiB5afMYg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_prime&d_ckId=34ce54af34ecf6404adb0908733c7c25&d_curPage=0&d_pageSize=40&d_headId=34ce54af34ecf6404adb0908733c7c25',
 '腾讯': '/zhaopin/?isAnalysis=&dqs=&pubTime=&salary=&compIds=7983148&subIndustry=&industryType=&compscale=&key=%E6%95%B0%E6%8D%AE%E6%8C%96%E6%8E%98&init=-1&searchType=1&headckid=96731afb8363a6ab&flushckid=1&compkind=&fromSearchBtn=2&sortFlag=15&ckid=96731afb8363a6ab&jobKind=&industries=&clean_condition=&siTag=LiAE77uh7ygbLjiB5afMYg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_prime&d_ckId=34ce54af34ecf6404adb0908733c7c25&d_curPage=0&d_pageSize=40&d_headId=34ce54af34ecf6404adb0908733c7c25',
 '字节跳动': '/zhaopin/?

### 使用 urllib.parse 解析


#### 检查相异值

In [143]:
df = pd.DataFrame([ urlparse(x) for x in company_url.values()])
df.nunique()# 发现query不同 

scheme       1
netloc       1
path         1
params       1
query       35
fragment     1
dtype: int64

In [145]:
#进一步检查query
df_qs = pd.DataFrame([{k:v[0] for k,v in parse_qs(x).items()} for x in df['query'] ])
df_qs.nunique()
#发现compIds不同

compIds          35
key               1
init              1
searchType        1
headckid          1
flushckid         1
fromSearchBtn     1
sortFlag          1
ckid              1
siTag             1
d_sfrom           1
d_ckId            1
d_curPage         1
d_pageSize        1
d_headId          1
dtype: int64

### 获得各公司专属ID

In [147]:
from urllib.parse import urlparse, parse_qs
#对url进行分解


import pprint
def parse_url_qs_for_compTag (url):
    six_parts = urlparse(url)    
    out = parse_qs(six_parts.query)
    return (out)

字典_compIds = { k:parse_url_qs_for_compTag(v)['compIds'][0] for k,v in company_url.items()}
字典_compIds
#df_qs
        
#e=list(company_url.values())
#for x in e:
#        pprint.pprint()
#k= [ parse_url_qs_for_compTag(x)   for x in e   ]
#k
#df_qs.nunique()

{'阿里巴巴': '1072424',
 '腾讯': '7983148',
 '字节跳动': '7863078',
 '美团点评': '2036768',
 '华为': '954482',
 '滴滴': '8076624',
 'Baidu': '884492',
 'vivo': '4787049',
 '饿了么': '2778803',
 '深圳商道名家教育科技有限公司': '9883043',
 '小米': '2174886',
 '京东数字科技控股有限公司': '215535',
 '携程': '182339',
 '亚信科技': '2399664',
 'OPPO移动通信': '3922321',
 '京东商城': '9747025',
 '上海寻梦信息技术有限公司': '8537928',
 '58同城': '591850',
 '深圳平安综合金融服务有限公司': '7956861',
 '网易集团': '5964833',
 '资采信息': '8830801',
 '苏宁易购': '2051657',
 '上海钧正网络科技有限公司': '8515562',
 '浩鲸云计算科技股份有限公司': '7939633',
 '猎聘招聘顾问团队': '12176713',
 '京东': '1663745',
 '蚂蚁金服(杭州)网络技术有限公司': '9364743',
 '北京中科特瑞科技有限公司': '9497282',
 '平安科技': '1086536',
 '便利蜂': '8983822',
 '新浪微博': '2638435',
 '毕马威中国 KPMG China': '8169921',
 '北京快手科技有限公司': '8605497',
 '中国移动通信有限公司研究院': '9122701',
 '好未来': '922091'}

###   参数模板生成

#### 设置参数的值，例如：keyword、key等

In [174]:
def 参数模板生成(compIds , keyword,key ):
    参数 = 参数模板.copy()
    参数['compIds'] = compIds
    参数['keyword'] = keyword
    参数['key']=key
    return (参数)

参数_配置url筛选 = { k:参数模板生成(compIds = [v],keyword = ['用户体验'],key=['产品经理']) for k,v in 字典_compIds.items()}
pprint.pprint(参数_配置url筛选)

{'58同城': {'ckid': ['970029fdeea53da8'],
          'compIds': ['591850'],
          'compTag': ['155'],
          'd_ckId': ['5b217fbd8e34ad6a68b83bd1bebc133f'],
          'd_curPage': ['0'],
          'd_headId': ['5b217fbd8e34ad6a68b83bd1bebc133f'],
          'd_pageSize': ['40'],
          'd_sfrom': ['search_prime'],
          'flushckid': ['1'],
          'fromSearchBtn': ['2'],
          'headckid': ['970029fdeea53da8'],
          'init': ['-1'],
          'key': ['产品经理'],
          'keyword': ['用户体验'],
          'searchType': ['1'],
          'siTag': ['LiAE77uh7ygbLjiB5afMYg~fA9rXquZc5IkJpXC-Ycixw'],
          'sortFlag': ['15']},
 'Baidu': {'ckid': ['970029fdeea53da8'],
           'compIds': ['884492'],
           'compTag': ['155'],
           'd_ckId': ['5b217fbd8e34ad6a68b83bd1bebc133f'],
           'd_curPage': ['0'],
           'd_headId': ['5b217fbd8e34ad6a68b83bd1bebc133f'],
           'd_pageSize': ['40'],
           'd_sfrom': ['search_prime'],
           'flushckid': 

### 配置请求url

In [156]:
#多个页面准备测试1 世界500强
url = "https://www.liepin.com/zhaopin/"
session = HTMLSession()
payload = 参数_配置url筛选['京东']
print(payload)
r = session.get( url, params = payload)
r.url

{'key': ['数据分析'], 'init': ['-1'], 'searchType': ['1'], 'headckid': ['970029fdeea53da8'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'compTag': ['155'], 'sortFlag': ['15'], 'ckid': ['970029fdeea53da8'], 'siTag': ['LiAE77uh7ygbLjiB5afMYg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_prime'], 'd_ckId': ['5b217fbd8e34ad6a68b83bd1bebc133f'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['5b217fbd8e34ad6a68b83bd1bebc133f'], 'compIds': ['1663745'], 'keyword': ['python']}


'https://www.liepin.com/zhaopin/?key=%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90&init=-1&searchType=1&headckid=970029fdeea53da8&flushckid=1&fromSearchBtn=2&compTag=155&sortFlag=15&ckid=970029fdeea53da8&siTag=LiAE77uh7ygbLjiB5afMYg~fA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_prime&d_ckId=5b217fbd8e34ad6a68b83bd1bebc133f&d_curPage=0&d_pageSize=40&d_headId=5b217fbd8e34ad6a68b83bd1bebc133f&compIds=1663745&keyword=python'

### 爬取信息函数

In [161]:
import pandas as pd
from requests_html import HTMLSession
session= HTMLSession()

#url="https://www.liepin.com/zhaopin/"

#爬取主函数
def requests(url,params):
    r= session.get(url,params = payload)
    #设置主要的xpath路径
    main= r.html.xpath("//div[contains(@class,'sojob-item-main')]")
    #使用字典存取特定的数据的位置
    dict_xpath={
        "text":{
            "职位":".//div[@class='job-info']/h3/a/text()"
        },
        "text_content":{
            "时间":".//div[@class='job-info']/p[contains(@class,'time-info')]/time/@title",

            "薪水":".//div[@class='job-info']/p/span[@class='text-warning']/text()",
            "工作地点":".//p[contains(@class,condition)]/a/text()",  #/div[@class='job-info']      [@class='area']
            "学历":".//span[@class='edu']/text()",
            "经验":".//p[contains(@class,condition)]/span[3]/text()",
            "公司名称":".//p[contains(@class,'company-name')]/a/text()"
        },
        "href":{
            "职位链接":".//div[@class='job-info']/h3/a",
            "公司链接":"//div[contains(@class,'sojob-item-main')]//p[@class='company-name']/a" #/div[contains(@class,'company-info')]
        }

    }

    def get_e_text(_xpath_):
            # 高级列表推导
            暂存结果 = ["".join([x.strip() for x in e.xpath(_xpath_)]) for e in main]
            return(暂存结果)

    def get_text_content(_xpath_):
            # 高级列表推导

            暂存结果 =[ e.xpath(_xpath_)[0]  for e in main]

            #暂存结果 =main#.xpath(_xpath_)
            return(暂存结果)          

        #如果为真，first=True返回第一个元素

    def get_href(_xpath_):          
            暂存结果=[list(e.xpath(_xpath_)[0].absolute_links)[0]              
                          if len(list(e.xpath(_xpath_)[0].absolute_links))>=1
                                else ""  for e in main  ]


            return(暂存结果)
    数据字典=dict()
    数据字典={k:get_text_content(v) for k,v in dict_xpath["text_content"].items()}
    数据字典.update({k:get_e_text(v) for k,v in dict_xpath['text'].items()})
    数据字典.update({k:get_href(v) for k,v in dict_xpath['href'].items() })
    数据字典
    df=pd.DataFrame(数据字典)
    return(df)

### 请求URL

####   多个页面,即多个公司

In [176]:
#配置请求筛选要求
def 参数模板生成(compIds , keyword,key ):
    参数 = 参数模板.copy()
    参数['compIds'] = compIds
    参数['keyword'] = keyword
    参数['key']=key
    return (参数)

参数_配置url筛选 = { k:参数模板生成(compIds = [v], keyword = ['用户体验'],key=['交互设计师']) for k,v in 字典_compIds.items()}
#pprint.pprint(参数_配置url筛选)

In [177]:
#   多个页面,即多个公司
import pandas as pd
url = "https://www.liepin.com/zhaopin/"
#payload = 参数_配置url筛选['阿里巴巴']
#DataFrame.assign(**kwargs)为DataFrame分配新列。返回一个新对象，该对象包含除新列之外的所有原始列。
list_ef = list()
#keyword为用户体验，key为数据，职位囊括上市公司，中国500强的公司的爬取
for k,v in 参数_配置url筛选.items():     
    payload = v
    df = requests( url, params = payload)#调用requests_liepin()函数
    df = df.assign(热门公司类型 = k)    
    list_df.append(df)
    
df_all_message = pd.concat(list_df)  #对数据的合并与重塑
df_all_message 

,时间,薪水,工作地点,学历,经验,公司名称,职位,职位链接,公司链接,热门公司类型
0,2020年04月03日,面议,杭州,学历不限,3年以上,阿里巴巴,新零售技术事业群-数据分析专家-躺平算法-淘系技术部,https://www.liepin.com/job/1927207743.shtml,https://www.liepin.com/company/1072424/,阿里巴巴
1,2020年04月03日,面议,杭州,学历不限,5年以上,阿里巴巴,阿里云智能事业群-销售数据分析专家-杭州,https://www.liepin.com/job/1927156871.shtml,https://www.liepin.com/company/1072424/,阿里巴巴
2,2020年04月03日,面议,杭州,学历不限,3年以上,阿里巴巴,新零售技术事业群-数据分析专家-淘系技术部,https://www.liepin.com/job/1921429517.shtml,https://www.liepin.com/company/1072424/,阿里巴巴
3,2020年04月02日,面议,杭州,学历不限,5年以上,阿里巴巴,集团公共事务部-数据分析专家-杭州,https://www.liepin.com/job/1927153713.shtml,https://www.liepin.com/company/1072424/,阿里巴巴
4,2020年04月02日,面议,杭州,学历不限,5年以上,阿里巴巴,集团公共事务部-数据分析专家-北京,https://www.liepin.com/job/1927153709.shtml,https://www.liepin.com/company/1072424/,阿里巴巴
...,...,...,...,...,...,...,...,...,...,...
4,2019年06月23日,12-18k·12薪,南京,大专及以上,5年以上,苏宁易购,UE设计师,https://www.liepin.com/job/1920933303.shtml,https://www.liepin.com/company/2051657/,苏宁易购
5,2019年09月20日,8-16k·12薪,南京-栖霞区,统招本科,2年以上,苏宁易购,交互设计师（智能硬件）,https://www.liepin.com/job/1920097181.shtml,https://www.liepin.com/company/2051657/,苏宁易购
6,2019年09月20日,15-20k·12薪,苏宁易购,统招本科,统招本科,苏宁易购,小程序前端开发工程师,https://www.liepin.com/job/1921761173.shtml,https://www.liepin.com/company/2051657/,苏宁易购
7,2019年04月02日,45-70k·14薪,南京,本科及以上,8年以上,苏宁易购,产品总监（广场方向）,https://www.liepin.com/job/1918756095.shtml,https://www.liepin.com/company/2051657/,苏宁易购


#### 单个页面个页面,即指定公司

In [167]:
#配置请求筛选要求
def 参数模板生成(compIds , keyword,key ):
    参数 = 参数模板.copy()
    参数['compIds'] = compIds
    参数['keyword'] = keyword
    参数['key']=key
    return (参数)

参数_配置url筛选 = { k:参数模板生成(compIds = [v], keyword = ['PRD'],key=['产品经理']) for k,v in 字典_compIds.items()}
#pprint.pprint(参数_配置url筛选)

In [168]:
import pandas as pd
url = "https://www.liepin.com/zhaopin/"
payload = 参数_配置url筛选['腾讯']
list_cf = list()
#keyword为用户体验，key为数据，职位囊括上市公司，中国500强的公司的爬取
cf = requests( url, params = payload)#调用requests_liepin()函数
cf = cf.assign(热门公司类型 = k)    
list_cf.append(cf)
    
df_one_message = pd.concat(list_cf)  #对数据的合并与重塑
df_one_message 

,时间,薪水,工作地点,学历,经验,公司名称,职位,职位链接,公司链接,热门公司类型
0,2020年04月05日,25-40k·12薪,深圳-南山区,统招本科,5年以上,腾讯,腾讯云生态产品经理,https://www.liepin.com/job/1927237017.shtml,https://www.liepin.com/company/7983148/,好未来
1,2020年04月04日,30-60k·12薪,深圳-南山区,统招本科,5年以上,腾讯,BI数据产品经理,https://www.liepin.com/job/1926393923.shtml,https://www.liepin.com/company/7983148/,好未来
2,2020年04月03日,35-55k·16薪,深圳,本科及以上,10年以上,腾讯,产品经理,https://www.liepin.com/job/1926042901.shtml,https://www.liepin.com/company/7983148/,好未来
3,2020年04月02日,面议,北京,本科及以上,5年以上,腾讯,CSIG16-垂类信息服务建设产品经理,https://www.liepin.com/job/1927177593.shtml,https://www.liepin.com/company/7983148/,好未来
4,2020年04月02日,面议,北京,本科及以上,5年以上,腾讯,30628-腾讯广告行业服务高级产品经理（北京）,https://www.liepin.com/job/1927177589.shtml,https://www.liepin.com/company/7983148/,好未来
5,2020年04月02日,面议,深圳,本科及以上,3年以上,腾讯,31967-企鹅电竞用户增长渠道产品经理,https://www.liepin.com/job/1927177563.shtml,https://www.liepin.com/company/7983148/,好未来
6,2020年04月02日,面议,深圳,本科及以上,2年以上,腾讯,32272-腾讯视频TV产品经理（深圳）,https://www.liepin.com/job/1927177551.shtml,https://www.liepin.com/company/7983148/,好未来
7,2020年04月02日,面议,深圳,本科及以上,2年以上,腾讯,36960-数据产品经理（北京或深圳）,https://www.liepin.com/job/1927177539.shtml,https://www.liepin.com/company/7983148/,好未来
8,2020年04月02日,面议,上海,本科及以上,2年以上,腾讯,30360-数据增长产品经理,https://www.liepin.com/job/1927177477.shtml,https://www.liepin.com/company/7983148/,好未来
9,2020年04月02日,面议,上海,本科及以上,5年以上,腾讯,19332-产品运营经理（上海）,https://www.liepin.com/job/1927177475.shtml,https://www.liepin.com/company/7983148/,好未来


## 分页输出爬取的数据

In [197]:
with pd.ExcelWriter('week03猎聘数据挖掘练习成果输出.xlsx' ,mode="w") as fp:
    df_all_message.to_excel(fp, sheet_name="交互设计师")
    数据挖掘.to_excel(fp, sheet_name="数据挖掘工程师")